In [2]:
import numpy as np

ResidualOffsetError = 53e-6
OffsetTempco = 4.8e-6
INLError = 128e-6
ResidualGainError = 80e-6
GainTempco = 11.3e-6
ReferenceTempco = 5e-6

TempChangeFromLastExternalCal = 5.
TempChangeFromLastInternalCal = 1.
Range = 20.

OutputValue = 1.

OffsetError = ResidualOffsetError + OffsetTempco * TempChangeFromLastInternalCal + INLError
GainError = ResidualGainError + GainTempco * TempChangeFromLastInternalCal + ReferenceTempco * TempChangeFromLastExternalCal

AbsoluteAccuracy = OutputValue * GainError + Range * OffsetError

In [157]:
import os
os.chdir('//share.ipp-hgw.mpg.de/documents/pih/Documents/git/QSB_Bolometry/libprad')

import matplotlib as mpl
import matplotlib.pyplot as p

from matplotlib.pyplot import cm
import numpy as np

import main as prad_main

data = prad_main.main(
    program='20181010.032',
    date=None,
    shot=None,
    POSIX_from=None,  # in ns
    POSIX_upto=None,  # in ns
    epoch_time=False,
    cont=False,
    filter_method='mean',
    geom_input='self',
    strgrid='sN8_30x20x150_1.3',
    magconf='EIM_beta000',
    plot=False,
    return_pow=False,
    Archive=False,
    compare_archive=False,
    versionize=False,
    ssp=False)


>> Start routine for P_rad calculation and diagnostics ...
	>> Selecting date ...
	>> Comparison info for shots ...
	>> Get info from logbook ...
	>> Current date is  20181010 ...
	>> SUCCESS: Server returns shot info of 20181010.032
	   NAME: #142598 ID07_O2_55MJ_9.1s_PrefillOnly
	   DESC.:  
	>> Get bolometer data and everything else...
		\\ timing error, mode: lab
		\\ 1.562%, new sample time:1.625ms
		\\ shift around T1/T4: 131.25 / 281.275 ms
		\\  <class 'TypeError'> webapi_access.py 109 stat  True  -- i --  1
	>> Finished check_bolo_data at 1
	>> Dump Run-Once stuff to priority_data...
		\\ using own geometry input: EIM_beta000 sN8_30x20x150_1.3
\\share.ipp-hgw.mpg.de\documents\pih\Documents\git\QSB_Bolometry\libprad
		>> Mean fitted calibration values are:
 			kappa_m= 6.128e-04 A^2; kappa_r= 6.167e-04 A^2
 			tau_m= 112.80 ms; tau_r= 110.47 ms;
 			roh_m= 996.86 Ohm; roh_r= 1007.60 Ohm;
	>> power and voltage: 0... 1... 2... 3... 4... 5... 7... 8... 9... 10... 11... 
		 12... 

In [158]:
prio = data['prio']
kappam = data['radpow']['kappam']
rohm = data['radpow']['rohm']
taum = data['radpow']['taum']

In [159]:
import prad_calculation
import magic_function

In [160]:
ch = 0
kappa_ch = kappam[ch]  # in A^2
r_ch = rohm[ch]  # in Ohm
tau_ch = taum[ch]  # in s
U0 = prio['U0']  # in V
rc = prio['RC']  # in Ohm
f_bridge = prio['f_bridge']  # in Hz
c_cab = prio['C_cab']  # in F
volume = prio['geometry']['geometry']['vbolo'][ch]  # in m^3
k_bolo = prio['geometry']['geometry']['kbolo'][ch]  # in m^3
volume_torus = prio['volume_torus']  # in m^3

In [161]:
U_eff = U0 * r_ch / (r_ch + 2.0 * rc)  # in V
omega = 2.0 * np.pi * f_bridge * c_cab  # F * Hz (Siemens)
rt_gc = np.sqrt(1.0 + (omega * (r_ch + rc))**2)  # unitless
beta = (1.0 - (omega * r_ch)**2 + (omega * rc)**2) / \
    (1.0 + (omega * (r_ch + rc))**2)  # unitless
tau_fact = 1.0 - U_eff**2 * beta / 4.0 / kappa_ch / \
    (r_ch + rc)**2  # unitless

# finally power
F =  2.0 / U_eff * (r_ch + 2.0 * rc) * kappa_ch * rt_gc

FV  = volume_torus * F * tau_fact / k_bolo
print(
    tau_fact, '\n',
    F, '\n',
    F * tau_fact, '\n',
    FV, '\n',
    FV * 1.e-3 / 1.e6, '\n',
    FV * 1.e-5 / 1.e6)

0.9931168998217441 
 0.34297427619492477 
 0.3406135498933103 
 141187874051.45236 
 141.18787405145238 
 1.4118787405145239


In [3]:
import time

class PID:
    """PID Controller
    """

    def __init__(self, P=0.2, I=0.0, D=0.0, current_time=None):

        self.Kp = P
        self.Ki = I
        self.Kd = D

        self.sample_time = 0.00
        self.current_time = current_time if current_time is not None else time.time()
        self.last_time = self.current_time

        self.clear()

    def clear(self):
        """Clears PID computations and coefficients"""
        self.SetPoint = 0.0

        self.PTerm = 0.0
        self.ITerm = 0.0
        self.DTerm = 0.0
        self.last_error = 0.0

        # Windup Guard
        self.int_error = 0.0
        self.windup_guard = 20.0

        self.output = 0.0

    def update(self, feedback_value, current_time=None):
        """Calculates PID value for given reference feedback
        .. math::
            u(t) = K_p e(t) + K_i \int_{0}^{t} e(t)dt + K_d {de}/{dt}
        .. figure:: images/pid_1.png
           :align:   center
           Test PID with Kp=1.2, Ki=1, Kd=0.001 (test_pid.py)
        """
        error = self.SetPoint - feedback_value

        self.current_time = current_time if current_time is not None else time.time()
        delta_time = self.current_time - self.last_time
        delta_error = error - self.last_error

        if (delta_time >= self.sample_time):
            self.PTerm = self.Kp * error
            self.ITerm += error * delta_time

            if (self.ITerm < -self.windup_guard):
                self.ITerm = -self.windup_guard
            elif (self.ITerm > self.windup_guard):
                self.ITerm = self.windup_guard

            self.DTerm = 0.0
            if delta_time > 0:
                self.DTerm = delta_error / delta_time

            # Remember last time and last error for next calculation
            self.last_time = self.current_time
            self.last_error = error

            self.output = self.PTerm + (self.Ki * self.ITerm) + (self.Kd * self.DTerm)

    def setKp(self, proportional_gain):
        """Determines how aggressively the PID reacts to the current error with setting Proportional Gain"""
        self.Kp = proportional_gain

    def setKi(self, integral_gain):
        """Determines how aggressively the PID reacts to the current error with setting Integral Gain"""
        self.Ki = integral_gain

    def setKd(self, derivative_gain):
        """Determines how aggressively the PID reacts to the current error with setting Derivative Gain"""
        self.Kd = derivative_gain

    def setWindup(self, windup):
        """Integral windup, also known as integrator windup or reset windup,
        refers to the situation in a PID feedback controller where
        a large change in setpoint occurs (say a positive change)
        and the integral terms accumulates a significant error
        during the rise (windup), thus overshooting and continuing
        to increase as this accumulated error is unwound
        (offset by errors in the other direction).
        The specific problem is the excess overshooting.
        """
        self.windup_guard = windup

    def setSampleTime(self, sample_time):
        """PID that should be updated at a regular interval.
        Based on a pre-determined sampe time, the PID decides if it should compute or return immediately.
        """
        self.sample_time = sample_time

In [ ]:
import time

import os
os.chdir('//share.ipp-hgw.mpg.de/documents/pih/Documents/git/QSB_Bolometry/libprad')

import matplotlib as mpl
import matplotlib.pyplot as p

from matplotlib.pyplot import cm
import numpy as np

from scipy.interpolate import BSpline, make_interp_spline

fig, ax = p.subplots(1, 3, sharey=True)

colors = ['r', 'b', 'g', 'purple']
L = 40
END = L

pd, id, dd = 1., .5, .003

for n, P in enumerate([pd, 0.3, .7, 1.2]):
    for j, I in enumerate([id, .0 , 5., 10.]):
        for l, D in enumerate([dd, .0, .0025, .004]):
            if (P == pd and I == id and D != dd) or (
                    P == pd and I != id and D == dd) or (
                    P != pd and I == id and D == dd):
                pass
            else:
                continue

            pid = PID(P, I, D)
            pid.SetPoint = 0.0
            pid.setSampleTime(0.001)

            feedback = 0
            feedback_list = np.zeros(L)
            time_list = np.linspace(.0, L, L)
            setpoint_list = np.zeros(L)

            for i in range(1, END):
                t = time_list[i]
                pid.update(feedback)
                output = pid.output

                if pid.SetPoint > 0:
                    feedback += (output - (1 / t))
                
                if t > 9:
                    pid.SetPoint = 1.

                time.sleep(0.01)
                feedback_list[i] = feedback
                setpoint_list[i] = pid.SetPoint

            time_smooth = np.linspace(
                time_list.min(), time_list.max(), 300)
            helper_x3 = make_interp_spline(time_list, feedback_list)
            feedback_smooth = helper_x3(time_smooth)

            if P != pd and I == id and D == dd:
                k, label, c = 0, 'K$_{p}$=' + format(P, '.1f'), colors[n - 1]
            elif I != id and P == pd and D == dd:
                k, label, c = 1, 'K$_{i}$=' + format(I, '.1f'), colors[j - 1]       
            elif D != dd and I == id and P == pd:
                k, label, c = 2, 'K$_{d}$=' + format(D, '.4f'), colors[l - 1]
            ax[k].plot(
                time_smooth, feedback_smooth,
                label=label, alpha=0.7, color=c)

for x in ax:
    x.plot(
        time_list, setpoint_list,
        c='k', ls='-.', alpha=0.5)
    #    label='SP')

    x.set_xlabel('time (s)')
    x.set_xlim(8., 30.)
    x.legend(
        # loc='center', bbox_to_anchor=(.55, .5), ncol=2,
        fancybox=True, shadow=False)

ax[0].set_ylabel('PID (PV, SP)')
ax[0].set_ylim(.25, 1.5)

ax[0].set_title('K$_{i}$=0.5, K$_{d}$=0.003')
ax[1].set_title('K$_{p}$=1.0, K$_{d}$=0.003')
ax[2].set_title('K$_{p}$=1.0, K$_{i}$=0.5')

fig.set_size_inches(8., 3.)
fig.savefig('pid_example.pdf', dpi=169.)

In [ ]:
import matplotlib.pyplot as p
import numpy as np
import camera_geometries as cgeom

import invert_main as invert
import os
os.chdir('//share.ipp-hgw.mpg.de/documents/pih/Documents/git/QSB_Bolometry/libprad')

(raw_volume, old_volume, volume, line_sections2D,
 line_sections3D, emissivity2D, emissivity3D,
 factors, reff, pos_lofs, minor_radius, reff_LoS,
 camera_geometry, interpolated_geometry, LoS) = invert.main(
        nPhi=30,
        nL=150,
        nFS=20,
        vpF=1.3,
        N=[2, 4],  # 4
        tilt_deg=1.,
        error_scale=0.0001,  # 0.1 mm (0.5 mm)
        VMID='EIM_000',
        interp_method='square',  # triang
        new_type=None,  # 'HBCm'
        cartesian=False,
        add_camera=False,
        artificial_HBCm=False,
        fix_LoS=False,
        centered=False,
        symmetric=False,
        tilt=False,
        random_error=False,
        plot=False,
        debug=False)

D = np.array([
    camera_geometry['values']['x'],
    camera_geometry['values']['y'],
    camera_geometry['values']['z']])
v = cgeom.v_norm(D[:, 31, 0] - D[:, 0, 0])[0]
a = np.rad2deg(cgeom.vector_angle(v, np.array([v[0], v[1], 0])))

In [8]:
import numpy as np
import os
os.chdir('//share.ipp-hgw.mpg.de/documents/pih/Documents/git/QSB_Bolometry/libprad')

import dat_lists as lists
import main as prad_main
import webapi_access as api
import prad_calculation as prad_calc

import mfr2D_matrix_gridtransform as transf

import matplotlib.pyplot as p
import matplotlib as mpl
from matplotlib import cm

from matplotlib import gridspec

def get_cam_info(
        geom_input='self',
        strgrid='sN8_30x20x150_1.3',
        magconf='EIM_beta000'):
    info = lists.geom_dat_to_json(
        saving=False,
        geom_input=geom_input,
        strgrid=strgrid,
        printing=False,
        magconf=magconf)
    return (info)

def get_prad(
    XPID='20181010.032'):
    transf.prepare_geometry_power(
        program=XPID,
        label='_EIM_beta000_sN8_30x20x150_1.3',
        strgrid='sN8_30x20x150_1.3',
        debug=False)

    data = prad_main.main(
        program=XPID,
        filter_method='mean',
        geom_input='self',
        strgrid='sN8_30x20x150_1.3',
        magconf='EIM_beta000',
        Archive=False)

    PRAD = np.array([
        data['radpow']['time'],
        data['radpow']['P_rad_hbc'],
        data['radpow']['P_rad_vbc']])
    CHORD = np.array([
        data['radpow']['time'],
        data['radpow']['volscaled']])
    REFF = data['prio']['geometry']['radius']['reff']
    RHO = data['prio']['geometry']['radius']['rho']
    KAPPA = data['radpow']['kappam']
    TAU = data['radpow']['taum']
    ROH = data['radpow']['rohm']
    POWER = data['radpow']['power']
    PRIO = data['prio']
    return (PRAD, CHORD, REFF, RHO, KAPPA, TAU, ROH, POWER, PRIO)

def get_params(
    XPID='20181010.032'):
    program_info, req = api.xpid_info(program=XPID)
    start = str(program_info['programs'][0]['from'])
    stop = str(program_info['programs'][0]['upto'])

    t0 = program_info['programs'][0]['trigger']['1'][0]  # in ns
    t4 = program_info['programs'][0]['trigger']['4'][0]  # in ns

    datalist = []

    namelist = [
        'BoloSignal PARLOG', 'T_e ECE core', 'T_e ECE out', 'n_e lint',  # 0 - 3
        'W_dia', 'n_e QTB vol2', 'T_e QTB vol2', 'ECRH',  # 4 - 7
        'BoloSingleChannelFeedback', 'BoloRealTime_P_rad',  # 8 - 9
        'EKS1 Bolo1', 'EKS1 Bolo2', 'QSQ Params',  # 10 - 12
        'Main valve BG011', 'Main valve BG031',  # 13 - 14
        ['DivAEF11', 'DivAEF21', 'DivAEF31', 'DivAEF41', 'DivAEF51',  
         'DivAEF10', 'DivAEF20', 'DivAEF30', 'DivAEF40', 'DivAEF50'],
        'QSQ Feedback AEH51', 'QSQ Feedback AEH30',  # 15 - 16
        'SingleChannel PARLOG', 'RealTime PARLOG',  # 17 - 18
        'AEH30 control', 'AEH30 Kp', 'AEH30 Ki', 'AEH30 Kd',  # 19 - 22
        'AEH51 control', 'AEH51 Kp', 'AEH51 Ki', 'AEH51 Kd']  # 23 - 26

    for name in namelist:
        if isinstance(name, list):
            sublist = []
            for second in name:
                foo = api.download_single(
                    api.download_link(name=second),
                    program_info=program_info,
                    start_POSIX=start, stop_POSIX=stop)
                sublist.append(np.array([
                    [(t - t0) / 1e9 for t in foo['dimensions']],
                    foo['values']]))

        else:
            foo = api.download_single(
                api.download_link(name=name),
                program_info=program_info,
                start_POSIX=start, stop_POSIX=stop)
            datalist.append(np.array([
                [(t - t0) / 1e9 for t in foo['dimensions']],
                foo['values']]))

    return (datalist, sublist, namelist)

def get_thomson_traces(
        XPID='20181010.032'):
    base_URI = 'http://archive-webapi.ipp-hgw.mpg.de/'
    base0 = 'Test/raw/W7X/QTB_Profile/volume_'
    base1 = '_DATASTREAM/V'
    base200 = '/1/ne_map/'
    base210 = '/0/Te_map/'
    base201 = '/1/ne_s/'
    base211 = '/0/Te_s/'

    program_info, req = api.xpid_info(program=XPID)
    start = str(program_info['programs'][0]['from'])
    stop = str(program_info['programs'][0]['upto'])

    t0 = program_info['programs'][0]['trigger']['1'][0]  # in ns
    t4 = program_info['programs'][0]['trigger']['4'][0]  # in ns

    N, T = [], []
    for volume in range(1, 17):
        n, t = np.zeros((2, 0)), np.zeros((2, 0))

        for version in range(21, 0, -1):
            link_ne = base_URI + base0 + str(volume) + \
                base1 + str(version) + base200
            foo = api.download_single(
                link_ne, program_info=program_info,
                start_POSIX=start, stop_POSIX=stop)
            if np.shape(foo['values'])[0] > 0:
                n = np.array([
                    [(s - t0) / 1e9 for s in foo['dimensions']], 
                    foo['values']])

                link_ne = base_URI + base0 + str(volume) + \
                    base1 + str(version) + base201
                ns = api.download_single(
                    link_ne, program_info=program_info,
                    start_POSIX=start, stop_POSIX=stop)['values']
                break

        for version in range(21, 0, -1):
            link_Te = base_URI + base0 + str(volume) + \
                base1 + str(version) + base210
            foo = api.download_single(
                link_Te, program_info=program_info,
                start_POSIX=start, stop_POSIX=stop)
            if np.shape(foo['values'])[0] > 0:
                t = np.array([
                    [(s - t0) / 1e9 for s in foo['dimensions']], 
                    foo['values']])

                link_Te = base_URI + base0 + str(volume) + \
                    base1 + str(version) + base211
                ts = api.download_single(
                    link_Te, program_info=program_info,
                    start_POSIX=start, stop_POSIX=stop)['values']
                break

        if np.shape(n)[1] > 0:
            N.append([str(volume), n, ns])
        if np.shape(t)[1] > 0:
            T.append([str(volume), t, ts])

    return (N, T)

def data_pass(
        datalist,
        divs,
        prad,
        XPID='20181010.032'):
    ecrh = data[7]
    wdia = data[4]
    te_in = data[1]
    te_out = data[2]
    ne_lin = data[3]
    
    singelch = data[8]
    rtprad = data[9]
    eks1 = data[10]
    eks2 = data[11]
    qparms = data[12]
    feedaeh51 = data[15]
    feedaeh30 = data[16]

    aeh30_params = data[19]
    aeh30_Kp = data[20]
    aeh30_Ki = data[21]
    aeh30_Kd = data[22]
    
    aeh51_params = data[23]
    aeh51_Kp = data[24]
    aeh51_Ki = data[25]
    aeh51_Kd = data[26]

    bg011, bg031 = data[13], data[14]

    program_info, req = api.xpid_info(program=XPID)
    start = str(program_info['programs'][0]['from'])
    stop = str(program_info['programs'][0]['upto'])

    t0 = program_info['programs'][0]['trigger']['1'][0]  # in ns
    t4 = program_info['programs'][0]['trigger']['4'][0]  # in ns

    divnames = [
        'DivAEF11', 'DivAEF21', 'DivAEF31', 'DivAEF41', 'DivAEF51',  
        'DivAEF10', 'DivAEF20', 'DivAEF30', 'DivAEF40', 'DivAEF50']

    missingU, missingD = 0, 0
    div, divU, divD = np.zeros((2, 0)), np.zeros((2, 0)), np.zeros((2, 0))
    divertors = []
    for d, divertor in enumerate(divs):
        if np.shape(divertor)[1] == 0:
            print('\\\ ', divnames[d], 'not found')
            if divnames[d][-1] == '1':
                missingU += 1
            elif divnames[d][-1] == '0':
                missingD += 1
            continue

        if np.shape(div)[1] == 0:
            div = np.zeros((2, len(divertor[0])))
            divU = np.zeros((2, len(divertor[0])))
            divD = np.zeros((2, len(divertor[0])))
            div[0], divU[0], divD[0] = divertor[0], divertor[0], divertor[0]

        lin = np.linspace(
            np.mean(divertor[1][:5]),
            np.mean(divertor[1][-5:]),
            np.shape(divertor)[1])
        divertor[1] += -lin

        if divnames[d][-1] == '1':
            divU[1][:np.shape(divertor)[1]] += divertor[1] * 1e-6
        elif divnames[d][-1] == '0':
            divD[1][:np.shape(divertor)[1]] += divertor[1] * 1e-6

        divertors.append([
            divertor[0], divertor[1] * 1.e-6,
            divnames[d].replace('Div', '')])

    div[1][:np.shape(divU[1])[0]] += ((5 - missingU) / 5) * divU[1]
    div[1][:np.shape(divD[1])[0]] += ((5 - missingD) / 5) * divD[1]
    
    lin = np.linspace(
        np.mean(div[1][:5]),
        np.mean(div[1][-5:]),
        np.shape(div)[1])
    div[1] += -lin

    M = 200
    dwdia_dt = np.array([
        wdia[0], np.convolve(
            1. / M * np.ones((M,)),
            np.gradient(wdia[1]) / (wdia[0, 1] - wdia[0, 0]),
            mode='same')])

    try:
        if np.shape(ecrh)[1] > np.shape(prad)[1]:
            T = ecrh[0]
            E = ecrh[1]
            P = np.interp(
                T, prad[0], (prad[1] + prad[2]) / 2.)
            RT = np.interp(
                T, rtprad[0], rtprad[1][0])
        else:
            T = prad[0]
            P = (prad[1] + prad[2]) / 2.
            RT = rtprad[1][0]
            E = np.interp(
                T, ecrh[0], ecrh[1])
    
    except Exception as e:
        T = P = RT = E = np.zeros(np.shape(ecrh)[1])

    indices = np.where((T > -.01) & (T < (t4 - t0) / 1.e9))[0]
    T = T[indices]
    P, E, RT = P[indices], E[indices], RT[indices]

    M = 1000
    F = (
        np.convolve(P, 1  / M * np.ones((M,)), mode='same') / 1e6) / (
        np.convolve(E, 1 / M * np.ones((M,)), mode='same') / 1e3)
    FRT = np.convolve(
        RT, 1  / M * np.ones((M,)), mode='same') / (
        np.convolve(E, 1 / M * np.ones((M,)), mode='same') / 1e3)

    indices = np.where((F < .0) | (F > 1.1))[0]  # or
    F[indices] = np.nan
    indices = np.where((FRT < .0) | (FRT > 1.1))[0]  # or
    FRT[indices] = np.nan

    frad = np.array([T, F])
    rtfrad = np.array([T, FRT])

    return (
        ecrh, wdia, te_in, te_out,
        ne_lin, div, divertors, dwdia_dt,
        frad, rtfrad, singelch, rtprad, eks1,
        eks2, qparms, feedaeh51, feedaeh30,
        bg011, bg031, aeh30_params, aeh30_Kp,
        aeh30_Ki, aeh30_Kd, aeh51_params,
        aeh51_Kp, aeh51_Ki, aeh51_Kd)

def get_cLines(
        XPID='20181010.032'):
    program_info, req = api.xpid_info(program=XPID)
    start = str(program_info['programs'][0]['from'])
    stop = str(program_info['programs'][0]['upto'])

    t0 = program_info['programs'][0]['trigger']['1'][0]  # in ns
    t4 = program_info['programs'][0]['trigger']['4'][0]  # in ns):

    names = [
        'CIII 5', 'CIII 19', 'CIII 24', 'CIII 29',  # 0 - 3
        'CIII 36', 'CIII 41', 'CIII 53', 'CIII 57']  # 4 - 7
    lines = []
    for name in names:
        foo = api.download_single(
            api.download_link(name=name),
            program_info=program_info,
            start_POSIX=start, stop_POSIX=stop)
        lines.append(np.array([
            [(t - t0) / 1e9 for t in foo['dimensions']],
            foo['values']]))
    return (lines, names)


def align_yaxis(ax1, v1, ax2, v2, y2max):
    """ adjust ax2 ylimit so that v2 in ax2 is aligned to v1 in ax1.
        y2max is the maximum value in your secondary plot. """
    _, y1 = ax1.transData.transform((0, v1))
    _, y2 = ax2.transData.transform((0, v2))
    inv = ax2.transData.inverted()
    _, dy = inv.transform((0, 0)) - inv.transform((0, y1 - y2))
    miny, maxy = ax2.get_ylim()

    scale = 1.
    while scale * (maxy + dy) < y2max * 1.02:
        scale += 0.07
    ax2.set_ylim(scale * (miny + dy), scale * (maxy + dy))

In [62]:
os.chdir('//share.ipp-hgw.mpg.de/documents/pih/Documents/git/QSB_Bolometry/libprad')

# XPID, t1, t2 = '20180920.029', -.1, 2.
# XPID, t1, t2 = '20180920.031', -.1, 11.
# XPID, t1, t2 = '20180920.032', -.1, 11.
# XPID, t1, t2 = '20181010.032', -.1, 10.
# XPID, t1, t2 = '20181016.016', -.1, 32.
# XPID, t1, t2 = '20180920.049', 4.2, 8.2
# XPID, t1, t2 = '20180725.044', -.1, 13.25
# XPID, t1, t2 = '20180809.013', -.1, 6.5
# XPID, t1, t2 = '20181015.023', -.1, 10.
XPID, t1, t2 = '20181011.012', -.1, 7.

program_info, req = api.xpid_info(program=XPID)
start = str(program_info['programs'][0]['from'])
stop = str(program_info['programs'][0]['upto'])

t0 = program_info['programs'][0]['trigger']['1'][0]  # in ns
t4 = program_info['programs'][0]['trigger']['4'][0]  # in ns

data, divs, names = get_params(XPID=XPID)
prad, chord, reff, rho, kappa, \
    tau, roh, power, prio = get_prad(XPID=XPID)

info = get_cam_info()

(ecrh, wdia, te_in, te_out,
 ne_lin, div, divertors, dwdia_dt,
 frad, rtfrad, singlech, rtprad, eks1,
 eks2, qparms, feedaeh51, feedaeh30,
 bg011, bg031, aeh30_params, aeh30_Kp,
 aeh30_Ki, aeh30_Kd, aeh51_params,
 aeh51_Kp, aeh51_Ki, aeh51_Kd) = data_pass(data, divs, prad, XPID=XPID)

ne_Q, Te_Q = get_thomson_traces(XPID=XPID)

c3_lines, c3_names = get_cLines(XPID=XPID)

		\\ loaded mesh2D  0.00MB
		\\ load line sections3D  (128, 64, 20, 150) 187.50MB
		\\ load emissivity3D  (128, 64, 20, 150) 187.50MB
		\\ load volume  (128, 64) 0.06MB
		\\ load positions  (129, 20, 150, 4) 11.81MB
		\\ load reff  (129, 64, 20, 150) 188.96MB
		\\ load minor radius

	>> P_rad calculation for output ...
		\\  <class 'TypeError'> webapi_access.py 109 stat  True  -- i --  1
		\\ using own geometry input: EIM_beta000 sN8_30x20x150_1.3
\\share.ipp-hgw.mpg.de\documents\pih\Documents\git\QSB_Bolometry\libprad
	>> saving: 20181011.012
		>> store at: ../../bolometer_mfr/chpow/20181011.012/20181011.012_EIM_beta000_sN8_30x20x150_1.3_chnpow_HBCm.dat
		>> store at: ../../bolometer_mfr/chpow/20181011.012/20181011.012_EIM_beta000_sN8_30x20x150_1.3_chnpow_VBCr.dat
		>> store at: ../../bolometer_mfr/chpow/20181011.012/20181011.012_EIM_beta000_sN8_30x20x150_1.3_chnpow_VBCl.dat

>> Start routine for P_rad calculation and diagnostics ...
	>> Selecting date ...
	>> Comparison info for shot

In [64]:
# chans = [65, 73, 78, 86, 61]
chans = [14, 16, 7, 3, 0]

rad, vol = prad_calc.calculate_prad(
    time=prad[0], power=power,
    volume=prio['geometry']['geometry']['vbolo'],
    k_bolo=prio['geometry']['geometry']['kbolo'],
    volume_torus=prio['volume_torus'], channels=chans,
    camera_list=chans, date=20181010, shotno=32,
    brk_chan=info['channels']['droplist'], camera_tag='none',
    saving=False, method='mean', debug=True)

p.close('all')
p.plot(ecrh[0][::100], ecrh[1][::100] / 1.e3, c='k')
p.plot(prad[0], np.mean(prad[1:], axis=(0)) / 1.e6, c='grey', ls='-.')

# p.plot(prad[0], power[70] * 20000, c='green')
# p.plot(rtprad[0], rtprad[1][0], c='r')
# p.plot(singlech[0], singlech[1], c='b')

p.plot(prad[0], (rad / 1.e6) / 1.3, c='orange')
# p.plot(aeh51_params[0], np.array(aeh51_params[1]), c='cyan')
p.xlim(-.1, 10.)

14 ... 16 ... 7 ... 3 ... 0 ... 14 ... 16 ... 7 ... 3 ... 0 ... done!


(-0.1, 10.0)

In [65]:
singlech = np.array([
    prad[0], power[70] * 20000])
rtprad = np.array([
    prad[0], (rad / 1.e6) / 1.3])

if np.shape(ecrh)[1] > np.shape(prad)[1]:
    T = ecrh[0]
    E = ecrh[1]
    P = np.interp(
        T, prad[0], (prad[1] + prad[2]) / 2.)
    RT = np.interp(
        T, rtprad[0], rtprad[1])
else:
    T = prad[0]
    P = (prad[1] + prad[2]) / 2.
    RT = rtprad[1][0]
    E = np.interp(
        T, ecrh[0], ecrh[1])

indices = np.where((T > .05) & (T < (t4 - t0) / 1.e9))[0]
T = T[indices]
P, E, RT = P[indices], E[indices], RT[indices]

M = 100
F = (
    np.convolve(P, 1  / M * np.ones((M,)), mode='same') / 1e6) / (
    np.convolve(E, 1 / M * np.ones((M,)), mode='same') / 1e3)
FRT = np.convolve(
    RT, 1  / M * np.ones((M,)), mode='same') / (
    np.convolve(E, 1 / M * np.ones((M,)), mode='same') / 1e3)

indices = np.where((F < .0) | (F > 1.3))[0]  # or
F[indices] = np.nan
indices = np.where((FRT < .0) | (FRT > 1.3))[0]  # or
FRT[indices] = np.nan

frad = np.array([T, F])
rtfrad = np.array([T, FRT])

In [66]:
# fig, ax = p.subplots(1, 2)
fig, ax = p.subplots(2, 1, sharex=True)

E, = ax[0].plot(
    ecrh[0], ecrh[1] / 1e3,
    c='k', label='ECRH')
ax[0].fill_between(
    ecrh[0][::100], (ecrh[1][::100] / 1e3) * 0.9,
    (ecrh[1][::100] / 1e3) * 1.1,
    color='grey', alpha=0.66)

wdiaax = ax[0].twinx()
errWdia = np.std(wdia[1][-100:]) / 1.e3
errdWdia = np.std(dwdia_dt[1][-100:]) / 1.e3

D, = ax[0].plot(
    dwdia_dt[0], dwdia_dt[1] / 1.e3,
    c='purple', label='dW$_{dia}/dt$')
if errdWdia / 1.e3 > 0.01:
    ax[0].fill_between(
        dwdia_dt[0][::100],
        dwdia_dt[1][::100] / 1.e3 - errdWdia,
        dwdia_dt[1][::100] / 1.e3 + errdWdia,
        color='purple', alpha=0.66)

W, = wdiaax.plot(
    wdia[0], wdia[1] / 1.e3,
    c='brown', ls='-.', label='W$_{dia}$')
if errdWdia / 1.e3 > 0.005:
    wdiaax.fill_between(
        wdia[0][::100], wdia[1][::100] / 1.e3 - errWdia,
        wdia[1][::100] / 1.e3 + errWdia,
        color='brown', alpha=0.66)
align_yaxis(
    ax[0], .0, wdiaax, .0,
    np.max(wdia[1][::100] / 1.e3))

H, = ax[1].plot(
    prad[0], prad[1] / 1e6,
    c='r', label='HBC')
V, = ax[1].plot(
    prad[0], prad[2] / 1e6,
    c='b', label='VBC')

stdC = np.std(prad[1][-100:]) * 1e-6
ax[1].fill_between(
    prad[0], (prad[1] / 1e6 - stdC) * .95,
    (prad[1] / 1e6) * 1.05,
    color='r', alpha=0.33)
stdC = np.std(prad[2][-100:]) * 1e-6
ax[1].fill_between(
    prad[0], (prad[2] / 1e6 - stdC) * .95,
    (prad[2] / 1e6 - stdC) * 1.05,
    color='b', alpha=0.33)

# S, = ax[1].plot(
#     singlech[0], singlech[1],  # [0],
#     c='g', ls='-.', label='P$_{pred}^{(2)}$')
# R, = ax[1].plot(
#     rtprad[0], rtprad[1],  # [0],
#     c='k', ls='-.', label='P$_{pred}^{(1)}$')
# 
# stdC = np.std(singlech[1][-100:])
# ax[1].fill_between(
#     singlech[0], (singlech[1] - stdC) * .95, # [0] - stdC) * .95,
#     (singlech[1] + stdC) * 1.05,
#     color='g', alpha=.33)
# stdC = np.std(rtprad[1][-100:])
# ax[1].fill_between(
#     rtprad[0], (rtprad[1] - stdC) * .95, # [0] - stdC) * .95,
#     (rtprad[1] + stdC) * 1.05,
#     color='k', alpha=.33)

# feedax = ax[2].twinx()
# control = aeh51_params  # / 1.e6  # / 1.e20
# Q, = ax[2].plot(
#     control[0], control[1],
#     c='r', label='y(t)')
# Q, = ax[2].plot(
#     # [1, 3, 10], [.5, 1., 1.],
#     [3., 16., 32.], [1.2, 1.2, 1.1],
#     c='grey', ls=':', lw=2., label='SP')

# P, = ax[2].plot(
#     # rtprad[0], rtprad[1][0],
#     control[0], control[1] / 1.e20,
#     c='r', ls='-.', label='y(t) ($n_{e}$ int.)')

# fb, name = feedaeh51, 'AEH51'
# if np.min(fb[1]) < .0:
#     fb[1] += -1. * np.min(fb[1])
# A, = feedax.plot(
#     fb[0],  # fb[1],
#     np.convolve(fb[1] * 1 / 150, np.ones(10,) / 10., mode='same'),
#     c='b', ls='-.', label=name)
# 
# param_ax = ax[3]
# # kd_ax = param_ax.twinx()
# q, Kp, Ki, Kd = aeh51_params, aeh51_Kp, \
#     aeh51_Ki, aeh51_Kd
# colors, i, f = ['k', 'r', 'b', 'g'], 0, [300., 50., 3.e3, 1.]
# Kl = []
# for x, L in zip(
#         [Kp, Ki, Kd],  # q],
#         ['K$_{p}$ / 300', 'K$_{i}$ / 50',  # ]):
#          'K$_{d}$ / 3$\\times10^{3}$']):  # , '$\\varepsilon(t)$', ]):
#     l, = param_ax.plot(x[0], x[1] / f[i], c=colors[i], label=L)
#     Kl.append(l)
#     i += 1

# l, = kd_ax.plot(Kd[0], Kd[1] / f[i], label='K$_{d}$', c=colors[i])
# Kl.append(l)
# align_yaxis(
#     param_ax, .0, kd_ax, .0,
#     np.max(Kd[1] / f[i]))

ax[0].set_ylabel('power [MW]')
# ax[0].set_xlabel('time [s]')
ax[0].legend(
    [E, W, D], [l.get_label() for l in [E, W, D]],
    loc='upper center', bbox_to_anchor=(.5, 1.35),
    shadow=False, ncol=3, handletextpad=0.3,
    labelspacing=0.5, handlelength=1.5)

wdiaax.set_ylabel('energy [MJ]')
ax[1].set_ylabel('power, P$_{rad}$ [MW]')
ax[1].legend(
    [H, V],  # S, R],
    [l.get_label() for l in [H, V]],  # S, R]],
    loc='upper center', bbox_to_anchor=(.5, 1.35), shadow=False,
    ncol=4, handletextpad=0.3, labelspacing=0.5, handlelength=1.5)

# ax[2].set_ylabel('PID [a.u.]')
# feedax.set_ylabel('valve [a.u.]')
# ax[2].legend(
#     [Q, A],
#     [l.get_label() for l in [Q, A]],
#     loc='upper center', bbox_to_anchor=(.5, 1.35),
#     shadow=False, ncol=3, handletextpad=0.3,
#     labelspacing=0.5, handlelength=1.5)
# 
# ax[3].set_ylabel('PID [a.u.]')
# ax[3].set_xlabel('time [s]')
# ax[3].legend(
#     Kl,  [l.get_label() for l in Kl],
#     loc='upper center', bbox_to_anchor=(.5, 1.35),
#     shadow=False, ncol=3, handletextpad=0.3,
#     labelspacing=0.5, handlelength=1.5)

ax[0].set_xlim(t1, t2)
# ax[0].set_xlim(-0.1, 32.)
# ax[1].set_xlim(4.2, 8.2)

# fig.set_size_inches(8., 2.5)
# fig.set_size_inches(6.,  4.)
fig.set_size_inches(5., 4.)
# fig.set_size_inches(5., 6.)

fig.tight_layout()
fig.savefig(XPID.replace('.', '_') + '_power_feedback.pdf', dpi=169)
p.close('all')

In [116]:
tmp = np.shape(ecrh)[1]

a, b = 1., 1.32
tmp0 = np.where((ecrh[0] > a) & (ecrh[0] < b))[0]
tmp1 = np.shape(tmp0)[0]
tmp2 = np.linspace(1., 1.4, tmp1)

SP, tSP = [], []
for i, t in enumerate(ecrh[0]):
    if a < t < b:
        tSP.append(t)
        SP.append(tmp2[i - tmp0[0]])
    # elif a < t < b:
    #     tSP.append(t)
    #    SP.append(1.4)

tmp3 = np.where((ecrh[0] > a) & (ecrh[0] < b))[0]
SP = SP / (ecrh[1][tmp3] / 1.e3)

p.plot(tSP, SP)

In [67]:
fig, ax = p.subplots(3, 1, sharex=True)
# fig, ax = p.subplots(4, 1, sharex=True)

# ne_volumes = [2, 12, 13]
# ne_volumes = [0, 9, 14]
# ne_volumes = [0, 9, 13, 14]
ne_volumes = [0, 9, 14]

# Te_volumes = [2, 12, 13]
# Te_volumes = [0, 7, 12]
# Te_volumes = [0, 6, 9, 12]
Te_volumes = [6, 9, 12]

# channels = [14, 16, 7, 3, 0]
channels = [61, 65, 73, 78, 86]

SC = 70

neax = ax[0]
tempax = ax[1]
neax.plot(
    ne_lin[0], ne_lin[1] / 1.e19,
    ls='-.', c='k', label='n$_{e}$ Int.')
neax.fill_between(
    ne_lin[0], ne_lin[1] / 1.e19 - 0.2,
    ne_lin[1] / 1.e19 + 0.2,
    color='grey', alpha=0.66)

if True:
    ind = np.where(te_in[0] > -0.1)[0]
    indT = np.where((te_in[0] > 11.) & (te_in[0] < 12.))[0]
    tempax.plot(
        te_in[0][ind], te_in[1][ind],  # - np.mean(te_in[1][indT]),
        c='k', label='ECE core')

    # ind = np.where(te_out[0] > -0.1)[0]
    # indT = np.where((te_out[0] > 11.) & (te_out[0] < 12.))[0]
    # tempax.plot(
    #     te_out[0][ind], te_out[1][ind],  # + np.mean(te_out[1][indT]),
    #     c='green', label='ECE out.')

M = 7
c_m = mpl.colors.LinearSegmentedColormap.from_list(
    'mycolors', ['red', 'blue'])
norm = mpl.colors.Normalize(vmin=0, vmax=len(ne_volumes))
s_m = cm.ScalarMappable(cmap=c_m, norm=norm)
s_m.set_array([])
for i, v in enumerate(ne_volumes):
    if ne_Q[v][2] == []:
        neax.plot(
            ne_Q[v][1][0],
            np.convolve(ne_Q[v][1][1], np.ones((M,)) / M, mode='same'),
            ls='-.', marker='.', markersize=3.,
            color=s_m.to_rgba(i), label='vol.' + ne_Q[v][0])
    else:
        neax.errorbar(
            ne_Q[v][1][0],
            np.convolve(ne_Q[v][1][1], np.ones((M,)) / M, mode='same'),
            yerr=np.mean(ne_Q[v][1][2]),
            ls='-.', marker='.', markersize=4.,
            color=s_m.to_rgba(i), label='vol.' + ne_Q[v][0])

if True:
    M = 5
    norm = mpl.colors.Normalize(vmin=0, vmax=len(Te_volumes))
    s_m = cm.ScalarMappable(cmap=c_m, norm=norm)
    s_m.set_array([])
    for i, v in enumerate(Te_volumes):
        ind = np.where(
            (Te_Q[v][1][1] > .0) &
            (Te_Q[v][1][1] < 4.))[0]
        if Te_Q[v][2] == []:
            tempax.plot(
                Te_Q[v][1][0][ind], # Te_Q[v][1][1][ind],
                np.convolve(Te_Q[v][1][1][ind], np.ones((M,)) / M, mode='same'),
                ls='-.', marker='.', markersize=3.,
                color=s_m.to_rgba(i), label='vol.' + Te_Q[v][0])
        else:
            tempax.errorbar(
                Te_Q[v][1][0][ind], # Te_Q[v][1][1][ind],
                np.convolve(Te_Q[v][1][1][ind], np.ones((M,)) / M, mode='same'),
                yerr=np.mean(Te_Q[v][1][2]),
                ls='-.', marker='.', markersize=4.,
                color=s_m.to_rgba(i), label='vol.' + Te_Q[v][0])

if False:
    norm = mpl.colors.Normalize(vmin=0, vmax=len(channels))
    s_m = cm.ScalarMappable(cmap=c_m, norm=norm)
    s_m.set_array([])
    chordax = ax[2]
    for i, ch in enumerate(channels):
        chordax.plot(
            chord[0], chord[1][ch] / 1.e3,
            label='ch.#' + str(ch),
            ls='-.', c=s_m.to_rgba(i))

# if SC not in channels:
#     chordax.plot(
#         chord[0], chord[1][SC] / 1.e3,
#         label='ch.#' + str(SC), c='k')

frad_div = ax[2]
if np.shape(div)[1] == 0:
    frad_div.plot(
        frad[0], frad[1],
        c='k', label='f$_{rad}$')
    # frad_div.plot(
    #     rtfrad[0], rtfrad[1],
    #     c='r', ls='-.', label='pred.')
    # frad_div.plot(
    #     # [-100., 100.], [0.9, 0.9],
    #     tSP[::100], SP[::100],
    #     c='grey', ls=':', lw=2., label='SP')
    frad_div.legend(ncol=3, fancybox=True, shadow=False)
    frad_div.set_ylabel('f$_{rad}$ [a.u.]')

else:
    fradax = frad_div.twinx()
    F, = fradax.plot(
        frad[0], frad[1],
        c='k', label='f$_{rad}$')
    # R, = fradax.plot(
    #     rtfrad[0], rtfrad[1],
    #     c='r', ls='-.', label='pred.')

    # fradax.plot(
    #     # [-100., 100.], [0.9, 0.9],
    #     tSP[::100], SP[::100],
    #     c='grey', ls=':', lw=2., label='SP')

    for i, divertor in enumerate(divertors):
        frad_div.plot(
            divertor[0], divertor[1],
            c='pink', ls='-.')
    stdDiv = np.std(div[1][-20:])
    P, = frad_div.plot(
        div[0], div[1],
        c='purple', label='P$_{div}$')
    frad_div.fill_between(
        div[0], div[1] - stdDiv, div[1] + stdDiv,
        color='purple', alpha=0.33)

    frad_div.legend(
        [F, P],  # R],
        [l.get_label() for l in [F, P]],  # , R]],
        loc='upper center', bbox_to_anchor=(.5, 1.35),
        ncol=3, fancybox=True, shadow=False,
        handletextpad=0.3, labelspacing=0.5, handlelength=1.)
    fradax.set_ylabel('f$_{rad}$ [a.u.]')
    frad_div.set_ylabel('heat load [MW]')

neax.legend(
    loc='upper center', bbox_to_anchor=(.5, 1.35),
    ncol=4, fancybox=True, shadow=False,
    handletextpad=0.3, labelspacing=0.5, handlelength=1.5)
neax.set_ylabel('density [10$^{19}$ m$^{-3}$]')

# chordax.set_ylabel('bright. [kWm$^{-3}$]')
# # chordax.set_ylim(-2., 420.)
# chordax.legend(
#     loc='upper center', bbox_to_anchor=(.5, 1.5),
#     ncol=3, fancybox=True, shadow=False,
#     handletextpad=0.3, labelspacing=0.5, handlelength=1.)

tempax.set_ylabel('temperature [keV]')
tempax.legend(
    loc='upper center', bbox_to_anchor=(.5, 1.35),
    ncol=4, fancybox=True, shadow=False,
    handletextpad=0.3, labelspacing=0.5, handlelength=1.5)

frad_div.set_xlabel('time [s]')
frad_div.set_xlim(t1, t2)

fig.set_size_inches(5., 5.5)
fig.tight_layout()
fig.savefig(XPID.replace('.', '_') + '_dens_chord_frad_div.pdf', dpi=169.)
p.close('all')

In [ ]:
fig, ax = p.subplots(1, 2)  # , sharex=True)

ac = []
for axis in ax:
    ac.append(axis.twinx())

M = 15
broken = info['channels']['droplist']
for i, cam in enumerate(['HBCm', 'VBC']): 
    channels = info['channels']['eChannels'][cam]

    T = chord[0][::M]
    R_list, C = [], []
    for ch in channels:
        if ch not in broken:
            R_list.append(rho[ch])
            C.append(chord[1][ch, ::M])
    R, C = np.array(R_list), np.array(C)
    R, T = np.meshgrid(R, chord[0][::M])

    CS = ax[i].contourf(
        T, R, C.transpose().clip(.0) / 1.e3,
        20, cmap=cm.viridis)

    cax = fig.add_axes([
        0.085 + i * .495, 1.02,
        0.355, 0.03])
    cb = fig.colorbar(
        CS, pad=0.1, cax=cax,  # ax=ax[i],
        orientation='horizontal')
    cb.ax.set_title('brightness [kWm$^{-3}$]')

    Nchannels = np.linspace(
        .0, len(channels) - 1, len(channels))
    tick_locs = ax[i].get_yticks()
    N = int(np.floor(
        np.shape(channels)[0] / np.shape(tick_locs)[0]))
    ac[i].set_yticks(R_list[::N])
    ac[i].set_yticklabels([str(int(ch)) for ch in Nchannels[::N]])

    ax[i].set_xlim(t1, t2)
    ax[i].set_xlabel('time [s]')
    ax[i].set_ylabel('radius [r$_{a}$]')
    ac[i].set_ylabel('channel [#]')

ax[0].text(.5, 0.85, 'HBC', color='white')
ax[1].text(.5, 0.73, 'VBC', color='white')

fig.set_size_inches(9., 3.)
fig.tight_layout()
fig.savefig(XPID.replace('.', '_') + '_chord_contour.pdf', dpi=169.)
p.close('all')

In [ ]:
fig, ax = p.subplots(1, 2, sharey=True)

ac = []
for axis in ax:
    ac.append(axis.twiny())

# times = [1., 1.25, 1.4, 1.55, 1.6]
# times = [1., 1.5, 1.8, 2.5]
# times = [1., 2., 3., 4.]
times = [0.62, 2.21, 7.15, 3.42]
times = [0.65, 1.65, 2.45, 3.05]

# times = [4.8, 4.9, 5., 5.1, 5.2]
# times = [1.75, 2.25, 5., 5.25]

T = np.array(chord[0])

r_m = mpl.colors.LinearSegmentedColormap.from_list(
    'mycolors', ['black', 'red'])
b_m = mpl.colors.LinearSegmentedColormap.from_list(
    'mycolors', ['black', 'blue'])
norm = mpl.colors.Normalize(vmin=0, vmax=len(times))

r_m = cm.ScalarMappable(cmap=r_m, norm=norm)
r_m.set_array([])

b_m = cm.ScalarMappable(cmap=b_m, norm=norm)
b_m.set_array([])

colors = [r_m, b_m]

broken = info['channels']['droplist']
for i, cam in enumerate(['HBCm', 'VBC']): 
    channels = info['channels']['eChannels'][cam]

    E, R, C = [], [], []
    for ch in channels:
        if ch not in broken:
            E.append(np.std(chord[1][ch, -200:]))
            R.append(rho[ch])
            C.append(chord[1][ch])
    E, R, C = np.array(E), np.array(R), np.array(C)

    for j, t in enumerate(times) :
        d = np.abs(T - t)
        index = np.where(d == np.min(d))[0][0]
        selC = np.mean(C[:, index - 5:index + 5], axis=(1))

        ax[i].errorbar(
            R, selC / 1.e3, yerr=E / 1.e3,
            c=colors[i].to_rgba(j), marker='.',
            markersize=4., alpha=0.75, ls='-.',
            label=format(t, '.2f') + 's')

        if j == 0:
            ac[i].set_xlim(np.min(R), np.max(R))
            ax[i].set_xlim(np.min(R), np.max(R))
            ac[i].set_xlabel('channel [#]')
            ax[i].set_xlabel('radius [r$_{a}$]')

            Nchannels = np.linspace(
                .0, len(channels) - 1, len(channels))
            tick_locs = ax[i].get_xticks()
            N = int(np.floor(
                np.shape(channels)[0] / np.shape(tick_locs)[0]) - 1)
            ac[i].set_xticks(R[::N])
            ac[i].set_xticklabels([str(int(ch)) for ch  in Nchannels[::N]])

ax[0].set_ylabel('brigh. [kWm$^{-3}$]')
ax[0].legend(ncol=2, fancybox=True, shadow=False)
ax[1].legend(ncol=2, fancybox=True, shadow=False)

ax[0].axvline(-1., ls=':', color='grey', lw=2., alpha=0.66)
ax[0].axvline(1., ls=':', color='grey', lw=2., alpha=0.66)
ax[1].axvline(-1., ls=':', color='grey', lw=2., alpha=0.66)

fig.set_size_inches(9., 3.)
fig.tight_layout()
fig.savefig(XPID.replace('.', '_') + '_chord.pdf', dpi=169.)
p.close('all')

In [ ]:
p.plot(prad[0], prad[1] / 1.e6)
p.plot(ecrh[0], ecrh[1] / 1.e3)
p.plot(dwdia_dt[0][::1000], dwdia_dt[1][::1000] / 1.e3)
p.plot(div[0], div[1])

In [ ]:
import tomogram_metrics as tm
import importlib
importlib.reload(tm)

# (aE, aH, aV, aW, adW, aDiv, power_balance,
#  tomo_balance) =  tm.power_balance_tomogram(
#  program=XPID, plot=False, passVals=False)

hb = np.array([prad[0], prad[1]])
vb = np.array([prad[0], prad[2]])
ab = np.array([prad[0], np.mean(prad[1:], axis=(0))])

# dL = [aE, adW, div, hb, vb, ab]
dL = [ecrh, dwdia_dt, div, hb, vb, ab]

N = 200
x1 = 0
for i, v in enumerate(dL):
    ind = np.where((v[0] > (t0 - t0) / 1.e9) & (v[0] < (t4 - t0) / 1.e9))[0]
    dL[i] = np.array([v[0][ind], v[1][ind]])
    x1 = ind[-1] - ind[0] if (ind[-1] - ind[0]) >= x1 else x1

int_list = np.linspace(.0, 1., x1)
for i, v in enumerate(dL):
    old_int = np.linspace(.0, 1., np.shape(v[0])[0])
    T = np.interp(int_list, old_int, v[0])
    dL[i] = np.array([
        T, np.convolve(np.interp(T, v[0], v[1]),
                        np.ones((N,)) / N, mode='same')])

#           ECRH            Div         dwdiadt          prad
pbv = dL[0][1] / 1.e3 - dL[2][1] - dL[1][1] / 1.e3 - dL[4][1] / 1.e6  # vbc
pbh = dL[0][1] / 1.e3 - dL[2][1] - dL[1][1] / 1.e3 - dL[3][1] / 1.e6  # hbc
pba = dL[0][1] / 1.e3 - dL[2][1] - dL[1][1] / 1.e3 - dL[5][1] / 1.e6  # both

In [ ]:
fig, ax = p.subplots()

for L, pbal, c in zip(
        ['P$_{b, HBC}$', 'P$_{b, VBC}$'], [pbh, pbv], ['r',  'b']):
    l, = ax.plot(
        dL[0][0][::1000], np.convolve(pbal[::1000], np.ones((5,)) / 5., mode='same'),  #  * 1.e-6,
        c=c, lw=0.75, alpha=0.75, label=L)

err_dWdia = np.std(dL[1][1][-100:])
totP = dL[0][1] * 1.1e-3 - dL[2][1] * 0.92 - \
    dL[5][1] * 0.95e-6 - (dL[1][1] + err_dWdia) * 1.e-3
totM = dL[0][1] * 0.9e-3 - dL[2][1] * 1.08 - \
    dL[5][1] * 1.05e-6 - (dL[1][1] - err_dWdia) * 1.e-3

ax.fill_between(
    dL[0][0][::1000],
    totP[::1000],  #  * 1.e-6,
    totM[::1000],  #  * 1.e-6,
    color='grey', alpha=0.5)

ax.legend(
    # loc='upper center', bbox_to_anchor=(.5, 1.4),
    ncol=2, fancybox=True, shadow=False,
    handletextpad=0.3, labelspacing=0.5, handlelength=1.)

ax.set_xlim(.0, 32.)
ax.set_ylabel('power [MW]')
ax.set_xlabel('time [s]')
fig.set_size_inches(5., 2.5)
fig.tight_layout()
fig.savefig(XPID.replace('.', '_') + '_power_balance.pdf', dpi=169.)
p.close('all')